# First order Nomoto model for ship maneuvering dynamics
> Implementation of the so called first order Nomoto model for ship maneuvering dynamics

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [simulation, dynamics, spring mass damper]
- image: images/nomoto.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

The first order Nomoto model can be written as:
$$ K \cdot \delta=r+T_{1} \cdot \dot{r}+T_{2} \cdot \ddot{r} $$
where $r$ is yaw rate with its time derivatives and $\delta$ is the rudder angle. $K$, $T_{1}$
 and $T_{1}$ are the coefficients describing the hydrodynamics of the ship.